In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from app.database import SessionLocal, engine, Base
from app.models import Fencer, Tournament, Bout
import datetime
from selenium.common.exceptions import NoSuchElementException

In [3]:
Base.metadata.drop_all(engine)

In [4]:
Base.metadata.create_all(engine)

In [5]:
from web_scrapping_helpers import FIEPage, parse_metadata, parse_pool, pool_to_db, tournament_to_db, parse_name, round_to_db

In [6]:
Scraper = FIEPage('epee', 's', 'A', 2)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\liamc\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [7]:
Scraper.open_competions().previous().age_group('s').comp_category('A').select_weapon('epee').select_gender('m').select_event_type().search()


In [8]:
metadata = Scraper.get_competitions()

In [9]:
a = set(prelim_data).union(set(prelim_data2))

NameError: name 'prelim_data' is not defined

In [41]:
len(a)

148

In [32]:
prelim_data.extend(prelim_data2), len(prelim_data)

(None, 212)

In [33]:
prelims = set(prelim_data)

In [37]:
prelims

{('AL DAWOOD Zakaria', 'KSA', '15', 'HUSEMANN Luca', 'GER', '8'),
 ('AL DAWOOD Zakaria', 'KSA', '5', 'DAGANI Elia', 'SUI', '15'),
 ('ALEXANIN Dmitriy', 'KAZ', '15', 'FENYVESI Csaba', 'HUN', '13'),
 ('ALHUSSAIN Ahmad Abbas I.', 'KSA', '15', 'SINGH Udaivir', 'IND', '14'),
 ('ALLEN Gregory', 'GBR', '13', 'BERKTOLD Samuel', 'AUT', '15'),
 ('ANDRASFI Tibor', 'HUN', '14', 'MATSUMOTO Ryu', 'JPN', '13'),
 ('ARMALEO Filippo', 'ITA', '15', 'JOHNSTON Calum', 'GBR', '12'),
 ('ARMALEO Filippo', 'ITA', '15', 'WANG Zijie', 'CHN', '13'),
 ('BAECKSTROEM Ian', 'SWE', '14', 'GORBACHUK Yevgeniy', 'UKR', '15'),
 ('BAKAY Sedat', 'GER', '15', 'STANKEVYCH Volodymyr', 'UKR', '14'),
 ('BERNARD Florian', 'BEL', '7', 'MATEVOSOV Artem', 'UKR', '15'),
 ('BESKIN Grigori', 'ISR', '3', 'PISKOVATSKOV Jonathan', 'USA', '15'),
 ('BITSCH Peter', 'GER', '15', 'FAVA Alex', 'FRA', '11'),
 ('BITSCH Peter', 'GER', '15', 'PISKOVATSKOV Jonathan', 'USA', '10'),
 ('BOBROV Ilian', 'EST', '9', 'JOHANIDES Lukas Jakub', 'SVK', '15'),


In [10]:
Scraper.next_round(prelim=True)

In [9]:
def parse_tournament(metadata, Scraper, session):
    comp_meta, link = metadata
    tournament = tournament_to_db(comp_meta, session)
    main = Scraper.open_window(link)
    pool_data = Scraper.to_results().to_pools().get_pools_data()
    for pool in pool_data:
        bouts = parse_pool(pool)
        pool_to_db(bouts, tournament, session)
    session.commit()
    Scraper.to_prelims()
    prelim_data = set(Scraper.get_prelim_data())
    try:
        Scraper.next_round_prelim()
    except NoSuchElementException:
        Scraper.next_round_prelim(option=True)
        
    prelim_data2 = set(Scraper.get_prelim_data())
    prelim_bouts = prelim_data.union(prelim_data2)
    print(len(prelim_bouts))
    if prelim_data != prelim_bouts:
        rounds = [256, 128, 64]
    else:
        rounds = [128, 64]
    prelim_bouts = list(prelim_bouts)
    print(len(prelim_bouts))
    pos = 0
    for rnd in rounds:
        num_bouts = rnd//2
        round_to_db(prelim_bouts[pos:pos+num_bouts], f"P{rnd}", tournament, session)
    """Prelim Parsing Logic"""
    Scraper.to_tableu()
    tableu_data = Scraper.get_de_data()
    for i in range(2):
        Scraper.next_round_de().next_round_de()
        next_round_data = Scraper.get_de_data()
        tableu_data.extend(next_round_data)
    rounds = [64, 32, 16, 8, 4, 2]
    pos = 0
    for rnd in rounds:
        num_bouts = rnd//2
        round_to_db(tableu_data[pos:pos+num_bouts], str(rnd), tournament, session)
        #print(f"Round {rnd}:\n{tableu_data[pos:pos+num_bouts]}")
        pos += num_bouts


    session.commit() 


In [10]:
session = SessionLocal
session.begin()

In [12]:
Scraper.get_pools_data()

[['POOL 1',
  'Referees: CIVIERO Ambre (SUI)',
  '1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  'V/M',
  'HS',
  'HR',
  'HS/HR',
  '1',
  'CZE',
  'CUPR Michal',
  'V/5',
  'D/2',
  'D/2',
  'V/5',
  'V/5',
  'D/4',
  '0.5',
  '23',
  '22',
  '1',
  '2',
  'SLO',
  'SOBOCAN Rok',
  'D/1',
  'D/3',
  'D/0',
  'D/1',
  'D/3',
  'D/2',
  '0',
  '10',
  '29',
  '-19',
  '3',
  'FRA',
  'NGUEFACK Lilian',
  'V/5',
  'V/5',
  'V/5',
  'D/2',
  'D/4',
  'D/2',
  '0.5',
  '23',
  '19',
  '4',
  '4',
  'USA',
  'PISKOVATSKOV Jona...',
  'V/3',
  'V/5',
  'D/1',
  'V/3',
  'D/2',
  'D/0',
  '0.5',
  '14',
  '17',
  '-3',
  '5',
  'HUN',
  'KOCH Mate Tamas',
  'D/4',
  'V/5',
  'V/3',
  'D/2',
  'V/5',
  'V/5',
  '0.67',
  '24',
  '16',
  '8',
  '6',
  'ESP',
  'CASADO Elias',
  'D/4',
  'V/5',
  'V/5',
  'V/3',
  'D/3',
  'D/4',
  '0.5',
  '24',
  '24',
  '0',
  '7',
  'POL',
  'ANTKIEWICZ Mateusz',
  'V/5',
  'V/4',
  'V/5',
  'V/5',
  'D/2',
  'V/5',
  '0.83',
  '26',
  '17',
  '9'],
 ['POO

In [11]:

parse_tournament(metadata[1], Scraper, session)

['CUPR', 'Michal']
['SOBOCAN', 'Rok']
['NGUEFACK', 'Lilian']
['PISKOVATSKOV', 'Jona...']
['KOCH', 'Mate', 'Tamas']
['CASADO', 'Elias']
['ANTKIEWICZ', 'Mateusz']
['KELLY', 'Christopher']
['MINOBE', 'Kazuyasu']
['SHEUM', 'Bron']
['HERZBERG', 'Fabian']
['SERTAY', 'Yerlik']
['JOHANIDES', 'Lukas', 'J...']
['VANAVICHITRA', 'Tadc...']
['REIN', 'Stephan']
['FAVA', 'Alex']
['FABREGAT', 'Angel']
['KIM', 'Jaewon']
['DERKSEN', 'Ruben']
['CACERES', 'CASTELLAN...']
['ELSOKKARY', 'Ahmad']
['BAY', 'COLOMBANO', 'Emm...']
['GALLAGHER', 'PELLETI...']
['KOSHMAN', 'Nikita']
['GONZALEZ', 'Alexander']
['BODOCZI', 'Nikolaus']
['GALLY', 'Aymerick']
['JEONG', 'Seyun']
['LUBIENIECKI', 'Wojciech']
['HOCH', 'Andre']
['GAROZZO', 'Enrico']
['RAZ', 'Ori']
['HOYER', 'Martin']
['KONGSTAD', 'Conrad', 'S...']
['ITO', 'Inochi']
['HASANOV', 'Ruslan']
['PAVLOV', 'Kiril']
['DERKACH', 'Andriy']
['VALLOSIO', 'Andrea']
['SCHANZLEH', 'Thom']
['LEE', 'Seunghyun']
['ELSAYED', 'Ahmed']
['HADDAD', 'Justin']
['KULOZIK', 'Julian']
['M

In [23]:
fencer = session.query(Fencer).filter_by(last_name='GROVER').first()

In [25]:
fencer.bout_win

[ELLIOTT GROVER 5 to ANDREA VALLOSIO 2,
 ELLIOTT GROVER 5 to CURTIS MCDOWALD 2,
 ELLIOTT GROVER 5 to PAVEL TYCHLER 4,
 ELLIOTT GROVER 5 to TORNIKE CHKHETIANI 0,
 ELLIOTT GROVER 15 to KENTARO MURAYAMA 11,
 ELLIOTT GROVER 15 to AXEL MATTSSON 14,
 ELLIOTT GROVER 15 to KENTARO MURAYAMA 11]

In [17]:
session.query(Fencer).filter_by(country = 'CAN').all()[0].last_name

'GALLAGHER PELLETI...'

In [9]:
with SessionLocal as session, session.begin():
    tourn_meta, link = comp
    tournament = tournament_to_db(tourn_meta, session)
    main = Scraper.open_window(link)
    pool_data = Scraper.to_results().to_pools().get_pools_data()
    for pool in pool_data:
        bouts = parse_pool(pool)
        pool_to_db(bouts, tournament, session)





In [ ]:
for fencer in test_cases:
    first_names = []
    last_names = []
    for name in fencer:
        if name.isupper():
            name = ''.join([name[0], name[1:].lower()])
            last_names.append(name)
        else: first_names.append(name)
    first_name = ' '.join(first_names)
    last_name = ' '.join(last_names)
    print(first_name, last_name)


In [20]:
# session = SessionLocal
# session.begin()
fencer_test = session.query(Fencer).filter_by(first_name = 'SAMUEL').all()

In [22]:
fencer_test[0]

SAMUEL UNTERHAUSER, GER